## Intro

[](http://)![understat.JPG](http://sergilehkyi.com/wp-content/uploads/2019/06/understat.jpg)

In this notebook I will describe the process of scraping data from web portal [understat.com](https://understat.com) that has a lot of statistical information about all games in top 5 European football leagues.

From [understat.com](https://understat.com) home page:

* Expected goals (xG) is the new revolutionary football metric, which allows you to evaluate team and player performance.

* In a low-scoring game such as football, final match score does not provide a clear picture of performance.

* This is why more and more sports analytics turn to the advanced models like xG, which is a statistical measure of the quality of chances created and conceded.

* Our goal was to create the most precise method for shot quality evaluation.

* For this case, we trained neural network prediction algorithms with the large dataset (>100,000 shots, over 10 parameters for each).

* On this site, you will find our detailed xG statistics for the top European leagues.

At this moment they have not only xG metric, but much more, that makes this site perfect for scraping statistical data about football games.




We start by importing libraries that will be used in this project:
* numpy - fundamental package for scientific computing with Python
* pandas - library providing high-performance, easy-to-use data structures and data analysis tools
* requests - is the only Non-GMO HTTP library for Python, safe for human consumption. (love this line from official docs :D)
* BeautifulSoup - a Python library for pulling data out of HTML and XML files.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup

## Website research and structure of data

On the home page we can notice that the site has data for 6 European Leagues:

![leagues.jpg](http://sergilehkyi.com/wp-content/uploads/2019/06/leagues.jpg)

*   La Liga
*   EPL
*   BundesLiga
*   Serie A
*   Ligue 1
*   RFPL

And we also see that the data collected is starting from season 2014/2015. Another notion we make is the structure of URL. It is '`https://understat.com/league'` + '`/name_of_the_league`' + '`/year_start_of_the_season`'

![seasons.jpg](http://sergilehkyi.com/wp-content/uploads/2019/06/seasons.jpg)

So we create global variables with this data to be able to select any of those.

In [43]:
# create urls for all seasons of all leagues
base_url = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

Next step is to understand where the data is located on the web-page. For this we open Developer Tools in Chrome, go to tab "Network", find file with data (in this case 2018) and check the "Response" tab. This is what we will get after running *requests.get(URL)*

![requests_response_1.jpg](http://sergilehkyi.com/wp-content/uploads/2019/06/requests_response_1.jpg)

After going through content of the web-page we find that the data is stored under "script" tag and it is JSON encoded. So we will need to find this tag, get JSON from it and convert it into Python readable data structure.

![requests_response_2.jpg](http://sergilehkyi.com/wp-content/uploads/2019/06/requests_response_2.jpg)

In [44]:
# Starting with latest data for Spanish league, because I'm a Barcelona fan
url = base_url+'/'+leagues[1]+'/'+seasons[7]
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")

# Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
scripts = soup.find_all('script')

# Check our <script> tags
# for el in scripts:
#   print('*'*50)
#   print(el.text)

### Working with JSON

We found that the data interesting us is stored in teamsData variable, after creating a soup of html tags it becomes just a string, so we find that text and extract JSON from it.

In [45]:
import json

string_with_json_obj = ''

# Find data for teams
for el in scripts:
    if 'teamsData' in str(el):
        string_with_json_obj = str(el).strip()
        
print(string_with_json_obj)

# strip unnecessary symbols and get only JSON data
ind_start = string_with_json_obj.index("('")+2
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

<script>
	var teamsData = JSON.parse('\x7B\x2271\x22\x3A\x7B\x22id\x22\x3A\x2271\x22,\x22title\x22\x3A\x22Aston\x20Villa\x22,\x22history\x22\x3A\x5B\x7B\x22h_a\x22\x3A\x22h\x22,\x22xG\x22\x3A0.80527000000000004131806008444982580840587615966796875,\x22xGA\x22\x3A0.8497090000000000475921524412115104496479034423828125,\x22npxG\x22\x3A0.80527000000000004131806008444982580840587615966796875,\x22npxGA\x22\x3A0.08854040000000000520952170290911453776061534881591796875,\x22ppda\x22\x3A\x7B\x22att\x22\x3A89,\x22def\x22\x3A20\x7D,\x22ppda_allowed\x22\x3A\x7B\x22att\x22\x3A247,\x22def\x22\x3A14\x7D,\x22deep\x22\x3A17,\x22deep_allowed\x22\x3A2,\x22scored\x22\x3A1,\x22missed\x22\x3A0,\x22xpts\x22\x3A1.16009999999999990905052982270717620849609375,\x22result\x22\x3A\x22w\x22,\x22date\x22\x3A\x222020\x2D09\x2D21\x2017\x3A00\x3A00\x22,\x22wins\x22\x3A1,\x22draws\x22\x3A0,\x22loses\x22\x3A0,\x22pts\x22\x3A3,\x22npxGD\x22\x3A0.7167296000000000777419018049840815365314483642578125\x7D,\x7B\x22h_a\x22\x3A\x2

Once we have gotten our JSON and cleaned it up we can convert it into Python dictionary and check how it looks (uncomment print statement to do that).

### Understanding data with Python

In [54]:
# convert JSON data into Python dictionary
data = json.loads(json_data)
print(data.keys())
print('='*50)
print(data['71'].keys())
print('='*50)
print(data['71']['id'])
print('='*50)
print(data['71']['title'])
print('='*50)
print(data['71']['history'][0])

# Print pretty JSON data to check out what we have there
# s = json.dumps(data, indent=4, sort_keys=True)
# print(s)

dict_keys(['71', '72', '74', '75', '76', '78', '80', '81', '82', '83', '86', '87', '88', '89', '92', '220', '228', '229', '238', '245'])
dict_keys(['id', 'title', 'history'])
71
Aston Villa
{'h_a': 'h', 'xG': 0.80527, 'xGA': 0.849709, 'npxG': 0.80527, 'npxGA': 0.0885404, 'ppda': {'att': 89, 'def': 20}, 'ppda_allowed': {'att': 247, 'def': 14}, 'deep': 17, 'deep_allowed': 2, 'scored': 1, 'missed': 0, 'xpts': 1.1601, 'result': 'w', 'date': '2020-09-21 17:00:00', 'wins': 1, 'draws': 0, 'loses': 0, 'pts': 3, 'npxGD': 0.7167296000000001}


If you want to check how the entire <code>data</code> looks, just uncomment respective lines. (it is commented for the sake of saving screen space and do not oversaturate the view of notebook).

When we start to research the <code>data</code> we understand that this is a dictionary of dictionaries of 3 keys: *`id`*, *`title`* and *`history`*. The first layer of dictionary uses ids as keys too.

Also from this we understand that *`history`* has data regarding every single match the team played in its own league (League Cup or Champions League games are not included).

We can gather teams names if go over the first layer dictionary.

In [55]:
# Get teams and their relevant ids and put them into separate dictionary
teams = {}
for id in data.keys():
    teams[id] = data[id]['title']

The *`history`* is the array of dictionaries where keys are names of metrics (read column names) and values are values, despite how tautological is that :D.

We understand that column names repeat over and over again so we add them to separate list. Also checking how the sample values look like.

In [56]:
# EDA to get a feeling of how the JSON is structured
# Column names are all the same, so we just use first element
columns = []
# Check the sample of values per each column
values = []
for id in data.keys():
    columns = list(data[id]['history'][0].keys())
    values = list(data[id]['history'][0].values())
    break

print(columns)
print(values)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']
['h', 0.80527, 0.849709, 0.80527, 0.0885404, {'att': 89, 'def': 20}, {'att': 247, 'def': 14}, 17, 2, 1, 0, 1.1601, 'w', '2020-09-21 17:00:00', 1, 0, 0, 3, 0.7167296000000001]


Found that Aston Villa has the id=71, so getting all the data for this team to be able to reproduce the same steps for all teams in the league.

In [58]:
av_data = []
for row in data['71']['history']:
    av_data.append(list(row.values()))

df = pd.DataFrame(av_data, columns=columns)
df.head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,h,0.80527,0.849709,0.80527,0.088540,"{'att': 89, 'def': 20}","{'att': 247, 'def': 14}",17,2,1,0,1.1601,w,2020-09-21 17:00:00,1,0,0,3,0.716730
1,a,2.03222,0.534675,2.03222,0.534675,"{'att': 307, 'def': 33}","{'att': 143, 'def': 24}",10,5,3,0,2.4631,w,2020-09-28 17:00:00,1,0,0,3,1.497545


Wualya! We have the data for all matches of Sevilla in season 2018-2019 within La Liga!

Now we want to do that for all Spanish teams. Let's loop through that bites baby!

In [9]:
# Getting data for all teams
dataframes = {}
for id, team in teams.items():
  teams_data = []
  for row in data[id]['history']:
    teams_data.append(list(row.values()))
    
  df = pd.DataFrame(teams_data, columns=columns)
  dataframes[team] = df
  print('Added data for {}.'.format(team))
  

Added data for Sevilla.
Added data for Real Sociedad.
Added data for Espanyol.
Added data for Getafe.
Added data for Atletico Madrid.
Added data for Rayo Vallecano.
Added data for Valencia.
Added data for Athletic Club.
Added data for Barcelona.
Added data for Real Madrid.
Added data for Levante.
Added data for Celta Vigo.
Added data for Real Betis.
Added data for Villarreal.
Added data for Eibar.
Added data for Alaves.
Added data for Leganes.
Added data for Girona.
Added data for Real Valladolid.
Added data for SD Huesca.


Now we have a dictionary of DataFrames where key is the name of the team and value is the DataFrame with all games of that team.

In [10]:
# Sample check of our newly created DataFrame
dataframes['Barcelona'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,h,3.26753,0.248353,3.26753,0.248353,"{'att': 118, 'def': 17}","{'att': 407, 'def': 13}",20,0,3,0,2.9009,w,2018-08-18 23:15:00,1,0,0,3,3.019177
1,a,1.20392,0.510742,1.20392,0.510742,"{'att': 163, 'def': 16}","{'att': 316, 'def': 15}",15,4,1,0,1.9865,w,2018-08-25 23:15:00,1,0,0,3,0.693178


### Manipulations to make data as in the original source

We can notice that here such metrics as PPDA and OPPDA (ppda and ppda_allowed) are represented as total amounts of attacking/defensive actions, but in the original table it is shown as coefficient. Let's fix that!

In [11]:
for team, df in dataframes.items():
    dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    dataframes[team]['ppda_att'] = dataframes[team]['ppda'].apply(lambda x: x['att'])
    dataframes[team]['ppda_def'] = dataframes[team]['ppda'].apply(lambda x: x['def'])
    dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    dataframes[team]['oppda_att'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att'])
    dataframes[team]['oppda_def'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['def'])
    
# And check how our new dataframes look based on Sevilla dataframe
dataframes['Sevilla'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def
0,a,3.34997,2.392390,3.34997,1.649760,"{'att': 252, 'def': 19}","{'att': 263, 'def': 20}",12,5,4,1,1.9830,w,2018-08-19 21:15:00,1,0,0,3,1.700210,13.263158,252,19,13.150000,263,20
1,h,1.97161,0.671429,1.97161,0.671429,"{'att': 262, 'def': 16}","{'att': 237, 'def': 26}",11,3,0,0,2.3331,d,2018-08-26 21:15:00,0,1,0,1,1.300181,16.375000,262,16,9.115385,237,26


Now we have all our numbers for every single game. Few magic moves and transformations and we will have our data ready.

In [12]:
frames = []
for team, df in dataframes.items():
    df['team'] = team
    frames.append(df)
    
full_stat = pd.concat(frames)
full_stat = full_stat.drop(['ppda', 'ppda_allowed'], axis=1)

In [13]:
full_stat.head(10)

,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def,team
0,a,3.349970,2.392390,3.349970,1.649760,12,5,4,1,1.9830,w,2018-08-19 21:15:00,1,0,0,3,1.700210,13.263158,252,19,13.150000,263,20,Sevilla
1,h,1.971610,0.671429,1.971610,0.671429,11,3,0,0,2.3331,d,2018-08-26 21:15:00,0,1,0,1,1.300181,16.375000,262,16,9.115385,237,26,Sevilla
2,a,0.784969,1.182950,0.784969,1.182950,2,5,0,1,0.9556,l,2018-09-02 18:45:00,0,0,1,0,-0.397981,12.120000,303,25,7.095238,149,21,Sevilla
3,h,1.082550,2.127170,1.082550,2.127170,20,4,0,2,0.6338,l,2018-09-16 18:45:00,0,0,1,0,-1.044620,4.115385,107,26,13.409091,295,22,Sevilla
4,a,3.401280,2.248470,3.401280,1.505320,13,9,6,2,2.1285,w,2018-09-23 10:00:00,1,0,0,3,1.895960,10.217391,235,23,17.800000,267,15,Sevilla
5,h,2.292620,1.553510,2.292620,1.553510,7,10,3,0,1.9024,w,2018-09-26 20:00:00,1,0,0,3,0.739110,17.647059,300,17,12.294118,209,17,Sevilla
6,a,1.985320,0.563865,1.242040,0.563865,2,2,3,1,2.5246,w,2018-09-29 16:30:00,1,0,0,3,0.678175,23.909091,263,11,4.676471,159,34,Sevilla
7,h,2.237000,0.895831,2.237000,0.895831,8,9,2,1,2.3711,w,2018-10-07 16:30:00,1,0,0,3,1.341169,11.000000,242,22,12.238095,257,21,Sevilla
8,a,1.898010,3.219480,1.898010,2.476200,8,8,2,4,0.6312,l,2018-10-20 18:45:00,0,0,1,0,-0.578190,10.321429,289,28,10.714286,225,21,Sevilla
9,h,1.884310,1.013060,1.884310,1.013060,9,5,2,1,2.0721,w,2018-10-28 18:45:00,1,0,0,3,0.871250,11.937500,191,16,18.166667,327,18,Sevilla


Also in the original table we have values of differences between expected metrics and real. Let's add those too.

In [14]:
full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

In [15]:
full_stat.head()

,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def,team,xG_diff,xGA_diff,xpts_diff
0,a,3.349970,2.392390,3.349970,1.649760,12,5,4,1,1.9830,w,2018-08-19 21:15:00,1,0,0,3,1.700210,13.263158,252,19,13.150000,263,20,Sevilla,-0.650030,1.392390,-1.0170
1,h,1.971610,0.671429,1.971610,0.671429,11,3,0,0,2.3331,d,2018-08-26 21:15:00,0,1,0,1,1.300181,16.375000,262,16,9.115385,237,26,Sevilla,1.971610,0.671429,1.3331
2,a,0.784969,1.182950,0.784969,1.182950,2,5,0,1,0.9556,l,2018-09-02 18:45:00,0,0,1,0,-0.397981,12.120000,303,25,7.095238,149,21,Sevilla,0.784969,0.182950,0.9556
3,h,1.082550,2.127170,1.082550,2.127170,20,4,0,2,0.6338,l,2018-09-16 18:45:00,0,0,1,0,-1.044620,4.115385,107,26,13.409091,295,22,Sevilla,1.082550,0.127170,0.6338
4,a,3.401280,2.248470,3.401280,1.505320,13,9,6,2,2.1285,w,2018-09-23 10:00:00,1,0,0,3,1.895960,10.217391,235,23,17.800000,267,15,Sevilla,-2.598720,0.248470,-0.8715


Original table

![full_table.JPG](http://sergilehkyi.com/wp-content/uploads/2019/06/full_table.jpg)

## Scraping data for all teams of all leagues of all seasons

Testing the flow before going full into the process

In [16]:
season_data = dict()
season_data[seasons[4]] = full_stat
print(season_data)
full_data = dict()
full_data[leagues[0]] = season_data
print(full_data)

{'2018':    h_a        xG       xGA    ...       xG_diff  xGA_diff  xpts_diff
0    a  3.349970  2.392390    ...     -0.650030  1.392390    -1.0170
1    h  1.971610  0.671429    ...      1.971610  0.671429     1.3331
2    a  0.784969  1.182950    ...      0.784969  0.182950     0.9556
3    h  1.082550  2.127170    ...      1.082550  0.127170     0.6338
4    a  3.401280  2.248470    ...     -2.598720  0.248470    -0.8715
5    h  2.292620  1.553510    ...     -0.707380  1.553510    -1.0976
6    a  1.985320  0.563865    ...     -1.014680 -0.436135    -0.4754
7    h  2.237000  0.895831    ...      0.237000 -0.104169    -0.6289
8    a  1.898010  3.219480    ...     -0.101990 -0.780520     0.6312
9    h  1.884310  1.013060    ...     -0.115690  0.013060    -0.9279
10   a  0.413773  0.740780    ...      0.413773  0.740780    -0.0614
11   h  3.953750  0.686451    ...      1.953750 -0.313549    -0.1104
12   h  1.403580  0.533210    ...      0.403580  0.533210    -0.8699
13   a  1.645390  1.02110

Putting all the previous code into loops to get all data.

In [17]:
full_data = dict()
for league in leagues:
  
  season_data = dict()
  for season in seasons:    
    url = base_url+'/'+league+'/'+season
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")

    # Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
    scripts = soup.find_all('script')
    
    string_with_json_obj = ''

    # Find data for teams
    for el in scripts:
        if 'teamsData' in el.text:
          string_with_json_obj = el.text.strip()

    # print(string_with_json_obj)

    # strip unnecessary symbols and get only JSON data
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    
    
    # convert JSON data into Python dictionary
    data = json.loads(json_data)
    
    # Get teams and their relevant ids and put them into separate dictionary
    teams = {}
    for id in data.keys():
      teams[id] = data[id]['title']
      
    # EDA to get a feeling of how the JSON is structured
    # Column names are all the same, so we just use first element
    columns = []
    # Check the sample of values per each column
    values = []
    for id in data.keys():
      columns = list(data[id]['history'][0].keys())
      values = list(data[id]['history'][0].values())
      break
      
    # Getting data for all teams
    dataframes = {}
    for id, team in teams.items():
      teams_data = []
      for row in data[id]['history']:
        teams_data.append(list(row.values()))

      df = pd.DataFrame(teams_data, columns=columns)
      dataframes[team] = df
      # print('Added data for {}.'.format(team))
      
    
    for team, df in dataframes.items():
        dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
        dataframes[team]['ppda_att'] = dataframes[team]['ppda'].apply(lambda x: x['att'])
        dataframes[team]['ppda_def'] = dataframes[team]['ppda'].apply(lambda x: x['def'])
        dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
        dataframes[team]['oppda_att'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att'])
        dataframes[team]['oppda_def'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['def'])
    
    frames = []
    for team, df in dataframes.items():
        df['team'] = team
        frames.append(df)
    
    full_stat = pd.concat(frames)
    full_stat = full_stat.drop(['ppda', 'ppda_allowed'], axis=1)
    
    full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
    full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
    full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']
    
    full_stat.reset_index(inplace=True, drop=True)
    season_data[season] = full_stat
  
  df_season = pd.concat(season_data)
  full_data[league] = df_season
  
data = pd.concat(full_data)
data.head()
  

h_a       xG    ...      xGA_diff  xpts_diff
Bundesliga 2014 0   h  2.57012    ...      0.198420    -0.6514
                1   a  1.50328    ...      0.307950     0.5143
                2   h  1.22987    ...      0.310166    -0.8412
                3   a  1.03519    ...      0.203118     1.1367
                4   h  3.48286    ...      0.402844    -0.0713

[5 rows x 27 columns]

Renaming index to have column names in the output file and removing unnecessary counter.

In [18]:
data.index = data.index.droplevel(2)
data.index = data.index.rename(names=['league','year'], level=[0,1])
data.head()

h_a       xG       xGA    ...      xG_diff  xGA_diff  xpts_diff
league     year                           ...                                  
Bundesliga 2014   h  2.57012  1.198420    ...      0.57012  0.198420    -0.6514
           2014   a  1.50328  1.307950    ...      0.50328  0.307950     0.5143
           2014   h  1.22987  0.310166    ...     -0.77013  0.310166    -0.8412
           2014   a  1.03519  0.203118    ...      1.03519  0.203118     1.1367
           2014   h  3.48286  0.402844    ...     -0.51714  0.402844    -0.0713

[5 rows x 27 columns]

## Exporting data to CSV file

In [19]:
data.to_csv('understat_per_game.csv')